In [1]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [2]:
import pandas as pd
import numpy as np
from PIL import Image 
dfnew=pd.read_csv('file1.csv')

In [3]:
dfnew['image'] = dfnew['path'].map(lambda x: np.asarray(Image.open(x).resize((450,600))))

In [4]:
print(dfnew['path'].isna().sum())  # Check for NaN values
print(dfnew[dfnew['path'].apply(lambda x: not isinstance(x, str))])  # Check for non-string values


0
Empty DataFrame
Columns: [Unnamed: 0, lesion_id, image_id, dx, dx_type, age, sex, localization, path, cell_type, cell_type_idx, image]
Index: []


**2nd** **TIME**

In [5]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [6]:
df=dfnew
features=df.drop(columns=['cell_type_idx'],axis=1)
target=df['cell_type_idx']

In [10]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.25,random_state=666)
tf.unique(x_train_o.cell_type.values)
print(x_train_o)



      Unnamed: 0    lesion_id      image_id     dx    dx_type   age      sex  \
3544        4583  HAM_0001668  ISIC_0027434     nv  follow_up  40.0   female   
6099        7956  HAM_0001801  ISIC_0033293     nv      histo  35.0   female   
3018        3867  HAM_0000608  ISIC_0025756     nv  follow_up  30.0   female   
7357        9599  HAM_0005257  ISIC_0033751     nv  consensus   NaN  unknown   
2931        3734  HAM_0007441  ISIC_0024548     nv  follow_up  60.0     male   
...          ...          ...           ...    ...        ...   ...      ...   
2878        3666  HAM_0001544  ISIC_0030875     nv  follow_up  50.0   female   
1950        2442  HAM_0004769  ISIC_0025250   vasc  consensus  30.0     male   
7597        9938  HAM_0007047  ISIC_0025957  akiec      histo  60.0   female   
1922        2407  HAM_0003480  ISIC_0033844   vasc  consensus  70.0   female   
6380        8336  HAM_0000025  ISIC_0024349     nv      histo  35.0     male   

         localization                  

In [27]:
import numpy as np
import ast

def convert_string_to_array(str_array):
    try:
        return np.array(ast.literal_eval(str_array), dtype=np.float32)
    except:
        return np.zeros((1, 1, 3))

x_train = np.array([convert_string_to_array(img_str) for img_str in x_train_o['image'].tolist()])
x_test = np.array([convert_string_to_array(img_str) for img_str in x_test_o['image'].tolist()])

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_test_mean) / x_test_std


C:\Users\PRAWAT4\AppData\Local\Temp\ipykernel_18616\2578750665.py:19: RuntimeWarning: invalid value encountered in divide
  x_train = (x_train - x_train_mean) / x_train_std
C:\Users\PRAWAT4\AppData\Local\Temp\ipykernel_18616\2578750665.py:20: RuntimeWarning: invalid value encountered in divide
  x_test = (x_test - x_test_mean) / x_test_std


In [28]:
from keras.utils import to_categorical

y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [29]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 999)
# Reshape image in 3 dimensions (height = 100, width = 125 , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(224, 224, 3))
x_test = x_test.reshape(x_test.shape[0], *(224, 224, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(224, 224, 3))

ValueError: cannot reshape array of size 15501 into shape (5167,224,224,3)

In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical



# Ensure the lengths of x_train and y_train are the same
if len(x_train) != len(y_train):
    raise ValueError("The number of images and labels must be the same.")

# Resize images to 224x224
x_train_resized = np.array([cv2.resize(img, (224, 224)) for img in x_train])
x_train = x_train_resized.reshape(-1, 224, 224, 3)

# Split the dataset into training and validation sets
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.1, random_state=999)

# Convert labels to categorical (one-hot encoding) if necessary
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_validate = to_categorical(y_validate, num_classes)

# Check the shapes
print("Shape of x_train:", x_train.shape)
print("Shape of x_validate:", x_validate.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_validate:", y_validate.shape)

# Your model code goes here
# ...


In [14]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load your dataset here
# Example (replace this with your actual data loading code)
# x_train = [load_your_images_here]  # This should be a list/array of images
# y_train = [load_your_labels_here]  # This should be a list/array of labels

# Ensure the lengths of x_train and y_train are the same
if len(x_train) != len(y_train):
    raise ValueError("The number of images and labels must be the same.")

# Resize images to 224x224
x_train_resized = np.array([cv2.resize(img, (224, 224)) for img in x_train])
x_train = x_train_resized.reshape(-1, 224, 224, 3)

# Split the dataset into training and validation sets
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.1, random_state=999)

# Convert labels to categorical (one-hot encoding) if necessary
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_validate = to_categorical(y_validate, num_classes)

In [15]:
mobile = tensorflow.keras.applications.mobilenet.MobileNet()

In [16]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

In [17]:
import tensorflow as tf

def change_model(model, new_input_shape=(None, 40, 40, 3), custom_objects=None):
    # Create a new model with the new input shape
    new_input = tf.keras.Input(shape=new_input_shape[1:])
    output = model(new_input)

    new_model = tf.keras.Model(inputs=new_input, outputs=output)

    # Copy weights from the old model to the new one
    for layer in new_model.layers:
        if 'input' not in layer.name:
            try:
                old_layer = model.get_layer(name=layer.name)
                layer.set_weights(old_layer.get_weights())
                print("Loaded layer {layer}".format(layer.name))
            except:
                print("Could not transfer weights for layer {layer}".format(layer.name))

    return new_model

# Adjust the input shape as per your requirement
new_model = change_model(mobile, new_input_shape=(None, 100, 125, 3))
new_model.summary()


Could not transfer weights for layer mobilenet_1.00_224
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 125, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functi  (None, 1000)              4253864   
 onal)                                                           
                                                                 
Total params: 4253864 (16.23 MB)
Trainable params: 4231976 (16.14 MB)
Non-trainable params: 21888 (85.50 KB)
_________________________________________________________________


In [18]:
from keras import Input
from keras.layers import Dense, Dropout
# Define the input layer
input_layer = Input(shape=(224, 224, 3))

# Connect the input layer to the new model
x = new_model(input_layer)

# Create a new dense layer for predictions
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

# Build the model
model = Model(inputs=input_layer, outputs=predictions)
# Continue with the rest of your code

In [19]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [20]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [22]:
from keras.optimizers import Adam
model.compile(Adam(learning_rate=0.01), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

In [23]:
class_weights={
    0: 1.0, # akiec
    1: 1.0, # bcc
    2: 1.0, # bkl
    3: 1.0, # df
    4: 3.0, # mel # Try to make the model more sensitive to Melanoma.
    5: 1.0, # nv
    6: 1.0, # vasc
}

In [26]:
from keras.src.preprocessing.image import ImageDataGenerator

batch_size = 16
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.12,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.12,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  

filepath = "model2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit(datagen.flow(x_train,y_train, batch_size=batch_size), 
                              class_weight=class_weights,
                    validation_data=(x_validate,y_validate),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=10, verbose=1,
                   callbacks=callbacks_list)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/remove_squeezable_dimensions/Squeeze defined at (most recent call last):
  File "C:\Users\PRAWAT4\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main

  File "C:\Users\PRAWAT4\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code

  File "C:\pythonprojectharsh\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\pythonprojectharsh\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\pythonprojectharsh\venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\pythonprojectharsh\venv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\PRAWAT4\AppData\Local\Programs\Python\Python39\lib\asyncio\base_events.py", line 601, in run_forever

  File "C:\Users\PRAWAT4\AppData\Local\Programs\Python\Python39\lib\asyncio\base_events

In [ ]:
from tensorflow.keras.models import load_model
model.save('model2.h5')